## Train Pipeline

In this notebook we demonstrate how to train using the video database. 

Requirements: 

<ul>
    <li> Path to OpenFace2.0 build/bin folder. This folder is created after OpenFace compilation. Please follow the instructions given <a href="https://github.com/TadasBaltrusaitis/OpenFace/wiki">here</a> to compile OpenFace2.0 on your machine.
     <li> Path to the POI specific videos to train the one-class novelity detection model. The video should be cleaned as per the specifications given in README.md
     <li> Path to save the output trained model.
</ul>

In [3]:
%reload_ext autoreload
%autoreload 2
%pylab inline
import os
import utils as u
import matplotlib.pyplot as plt
import time
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

from IPython.display import clear_output

Populating the interactive namespace from numpy and matplotlib


### 1) Initialize


#### Hyper-parameters
There are few hyper-parameters that we need to set before training. The value of these parameters are currently set to the one used in the paper.pdf. 

<ul>
    <li> clip_length: length in number of frames over which the correlation is computed. Currently set to 300 which corresponds to 10-second length for a video saved at 30 fps.
    <li> shift_win: the number of frames to shift to extract the next overlapping video clip. 
    <li> c_t: the confidence threshold below which the feature estimated by OpenFace are rejected
</ul>

#### Paths

<ul>
    <li> open_face_path: path to OpenFace build/bin folder.
    <li> videos_path: the path to all the .mp4 videos for a POI on which we train the SVM model.
    <li> model_out_path: the output path to store the model file.
</ul>

In [6]:
#hyper parameters
clip_len = 300 #length of the clip in frames, assume the fps=30
shift_win = 5 #the number of frames to shift
c_t = 0.93 #the confidence threshold for OpenFace detection. Even a single less confident frame will reject the entire 10-second clip

#paths
open_face_path = 'OpenFace' #path to OpenFace build/bin folder
videos_path = '/Users/shruti_agarwal/Dropbox/data/projects/deep_fakes/videos/bo' #the path to all the .mp4 videos for a POI on which we train the SVM model
model_out_path = 'models/bo' # the output path to store the model file

### 2) Select the features

By default we train the model using the entire 190-D correlation features. The names of all the pairs of features can be extracted using u.get_all() function. 

We can also train using only selective features. For example, in case of Obama we found top 29 features to train the model. The names of these features are stored in the .csv file

In [5]:
n = 190 #number of features used, for Obama use n=29 features
feat_file = 'models/ablation_obama_bo_smalldiffbo_UWfakebo_faceswapbo_imposter.csv'

if n == 190:
    feat_nm = u.get_all()
else:
    feat_nm = u.get_feat_from_file(feat_file, n)
    
# get the correlation feature names
feat_nm = np.array([''.join(x) for x in feat_nm])

print(feat_nm)

['AU01_rAU02_r' 'AU01_rAU04_r' 'AU01_rAU05_r' 'AU01_rAU06_r'
 'AU01_rAU07_r' 'AU01_rAU09_r' 'AU01_rAU10_r' 'AU01_rAU12_r'
 'AU01_rAU14_r' 'AU01_rAU15_r' 'AU01_rAU17_r' 'AU01_rAU20_r'
 'AU01_rAU23_r' 'AU01_rAU25_r' 'AU01_rAU26_r' 'AU01_rpose_Rx'
 'AU01_rpose_Rz' 'AU01_rlip_ver' 'AU01_rlip_hor' 'AU02_rAU04_r'
 'AU02_rAU05_r' 'AU02_rAU06_r' 'AU02_rAU07_r' 'AU02_rAU09_r'
 'AU02_rAU10_r' 'AU02_rAU12_r' 'AU02_rAU14_r' 'AU02_rAU15_r'
 'AU02_rAU17_r' 'AU02_rAU20_r' 'AU02_rAU23_r' 'AU02_rAU25_r'
 'AU02_rAU26_r' 'AU02_rpose_Rx' 'AU02_rpose_Rz' 'AU02_rlip_ver'
 'AU02_rlip_hor' 'AU04_rAU05_r' 'AU04_rAU06_r' 'AU04_rAU07_r'
 'AU04_rAU09_r' 'AU04_rAU10_r' 'AU04_rAU12_r' 'AU04_rAU14_r'
 'AU04_rAU15_r' 'AU04_rAU17_r' 'AU04_rAU20_r' 'AU04_rAU23_r'
 'AU04_rAU25_r' 'AU04_rAU26_r' 'AU04_rpose_Rx' 'AU04_rpose_Rz'
 'AU04_rlip_ver' 'AU04_rlip_hor' 'AU05_rAU06_r' 'AU05_rAU07_r'
 'AU05_rAU09_r' 'AU05_rAU10_r' 'AU05_rAU12_r' 'AU05_rAU14_r'
 'AU05_rAU15_r' 'AU05_rAU17_r' 'AU05_rAU20_r' 'AU05_rAU23_r'
 'AU05_rAU25

### 3) extract the features

For all the videos in the train folder extract the above selected correlation features.

In [ ]:
#all the videos
vid_nms = [os.path.join(videos_path, f) for f in os.listdir(videos_path) if f.endswith('.mp4')]

#get all the features in a dictionary
features = {}
for v in vid_nms:

    #OpenFace
    face_features = u.get_facial_features(v, open_face_path, c_t)
    
    #Correlation
    features[v] = u.get_corr_per_frame(face_features, clip_len, shift_win)[feat_nm]
    
X_train = pd.concat(list(features.values()), ignore_index=True, sort=False)

print('train length {}'.format(len(X_train)))

### 4) train the model

Train the one class SVM model using the above computed features. The model is trained using rbf kernel with default values for gamma and nu. 

In [10]:
#train SVM
print('begin training...')
start = time.time()
model = u.train_ovr(np.array(X_train))
end_time = time.time()
print('done training in {} sec'.format(end_time-start))

#score range for traininf data
pred_prob = model['model'].score_samples(model['scaler'].transform(np.array(X_train)))
prob_rng = [np.quantile(pred_prob, q=0.02), np.quantile(pred_prob, q=0.98)] # this range is used to normalize the test scores
print('Score range min: {} max: {}'.format(prob_rng[0], prob_rng[1]))

#save the model with all the parameters to be used for testing
model['prob_rng'] = prob_rng
model['clip_len'] = clip_len
model['feat_name'] = feat_nm
model['shift_win'] = shift_win
model['c_t'] = c_t

#save the model
u.save_obj(model, os.path.dirname(model_out_path), os.path.basename(model_out_path))

begin training...
done training in 369.75785088539124 sec
Score range min: 8.961812240713925 max: 28.077287679223335
